In [ ]:
"""
There are some national statistics that will come in handy for the
Nobel Prize visualization we’re using our toolchain to build. Population
sizes, three-letter international codes (e.g., GDR, USA), and geographic
centers are potentially useful when you are visualizing an international
prize and its distribution. REST countries is a handy RESTful web resource
with various international stats. Let’s use it to grab some data.

Requests to REST countries take the following form:
https://restcountries.eu/rest/v1/<field>/<name>?<params>
"""

In [1]:
import requests
from pymongo import MongoClient

In [2]:
REST_EU_ROOT_URL = "http://restcountries.eu/rest/v1"

In [3]:
def REST_country_request(field='all', name=None, params=None):
    """
    It’s usually a good idea to specify a valid User-Agent in the
    header of your request. Some sites will reject the request otherwise.
    """
    headers={'User-Agent': 'Mozilla/5.0'}

    if not params:
        params = {}
    if field == 'all':
        return requests.get(REST_EU_ROOT_URL + '/all')

    url = '%s/%s/%s'%(REST_EU_ROOT_URL, field, name)
    print('Requesting URL: ' + url)
    response = requests.get(url, params=params, headers=headers)

    """
    Before returning the response, make sure it has an OK (200)
    HTTP code; otherwise, raise an exception with a helpful message.
    """
    if not response.status_code == 200:
        raise Exception('Request failed with status code ' + str(response.status_code))
    return response

In [ ]:
"""
With the REST_country_request function in hand, let’s get a list of
all the countries using the US dollar as currency:
"""

In [4]:
response = REST_country_request('currency', 'usd')
response.json()

Requesting URL: http://restcountries.eu/rest/v1/currency/usd


[{'name': 'American Samoa',
  'topLevelDomain': ['.as'],
  'alpha2Code': 'AS',
  'alpha3Code': 'ASM',
  'callingCodes': ['1684'],
  'capital': 'Pago Pago',
  'altSpellings': ['AS', 'Amerika Sāmoa', 'Amelika Sāmoa', 'Sāmoa Amelika'],
  'region': 'Oceania',
  'subregion': 'Polynesia',
  'population': 55519,
  'latlng': [-14.33333333, -170.0],
  'demonym': 'American Samoan',
  'area': 199.0,
  'gini': None,
  'timezones': ['UTC-11:00'],
  'borders': [],
  'nativeName': 'American Samoa',
  'numericCode': '016',
  'currencies': ['USD'],
  'languages': ['en', 'sm'],
  'translations': {'de': 'Amerikanisch-Samoa',
   'es': 'Samoa Americana',
   'fr': 'Samoa américaines',
   'ja': 'アメリカ領サモア',
   'it': 'Samoa Americane'},
  'relevance': '0.5'},
 {'name': 'Bonaire',
  'topLevelDomain': ['.an', '.nl'],
  'alpha2Code': 'BQ',
  'alpha3Code': 'BES',
  'callingCodes': ['5997'],
  'capital': 'Kralendijk',
  'altSpellings': ['BQ', 'Boneiru'],
  'region': 'Americas',
  'subregion': 'Caribbean',
  'popula

In [ ]:
"""
The full dataset at REST countries is pretty small, so for convenience
we’ll make a copy and store it locally to MongoDB and our nobelprize
database using the get_mongo_database method below:
"""

In [5]:
def get_mongo_database(db_name,
                       host='localhost', port=27017,
                       username=None, password=None):
    """ 
    Get named database from MongoDB with/out authentication
    --------------------------------------------------------
    We specify the database name in the MongoDB URI (Uniform
    Resource Identifier) as the user may not have general privileges
    for the database.
    """
    # make Mongo connection with/out authentication
    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s/%s'%(username, password, host, db_name)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)
    return conn[db_name]

In [6]:
def delete_collection(db_name, 
                      collection_name,
                      host='localhost',
                      port=27017,
                      username=None,
                      password=None):
    db = get_mongo_database(db_name, host, port, username, password)
    db[collection_name].delete_many({}) # empty filter deletes all entries

In [7]:
delete_collection('nobel_prize', 'country_data')

In [8]:
db_nobel = get_mongo_database('nobel_prize')
col = db_nobel['country_data']  # country data collection

In [9]:
# Get all the RESTful country-data
response = REST_country_request()
# Insert the JSON-objects straight to our collection
col.insert(response.json())

<ipython-input-9-986e2c524f4f>:4: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  col.insert(response.json())


[ObjectId('60bcb42e165a6db004093d51'),
 ObjectId('60bcb42e165a6db004093d52'),
 ObjectId('60bcb42e165a6db004093d53'),
 ObjectId('60bcb42e165a6db004093d54'),
 ObjectId('60bcb42e165a6db004093d55'),
 ObjectId('60bcb42e165a6db004093d56'),
 ObjectId('60bcb42e165a6db004093d57'),
 ObjectId('60bcb42e165a6db004093d58'),
 ObjectId('60bcb42e165a6db004093d59'),
 ObjectId('60bcb42e165a6db004093d5a'),
 ObjectId('60bcb42e165a6db004093d5b'),
 ObjectId('60bcb42e165a6db004093d5c'),
 ObjectId('60bcb42e165a6db004093d5d'),
 ObjectId('60bcb42e165a6db004093d5e'),
 ObjectId('60bcb42e165a6db004093d5f'),
 ObjectId('60bcb42e165a6db004093d60'),
 ObjectId('60bcb42e165a6db004093d61'),
 ObjectId('60bcb42e165a6db004093d62'),
 ObjectId('60bcb42e165a6db004093d63'),
 ObjectId('60bcb42e165a6db004093d64'),
 ObjectId('60bcb42e165a6db004093d65'),
 ObjectId('60bcb42e165a6db004093d66'),
 ObjectId('60bcb42e165a6db004093d67'),
 ObjectId('60bcb42e165a6db004093d68'),
 ObjectId('60bcb42e165a6db004093d69'),
 ObjectId('60bcb42e165a6d

In [10]:
res = col.find({'currencies':{'$in':['USD']}})
list(res)[0:2]

[{'_id': ObjectId('60bcb42e165a6db004093d55'),
  'name': 'American Samoa',
  'topLevelDomain': ['.as'],
  'alpha2Code': 'AS',
  'alpha3Code': 'ASM',
  'callingCodes': ['1684'],
  'capital': 'Pago Pago',
  'altSpellings': ['AS', 'Amerika Sāmoa', 'Amelika Sāmoa', 'Sāmoa Amelika'],
  'region': 'Oceania',
  'subregion': 'Polynesia',
  'population': 55519,
  'latlng': [-14.33333333, -170.0],
  'demonym': 'American Samoan',
  'area': 199.0,
  'gini': None,
  'timezones': ['UTC-11:00'],
  'borders': [],
  'nativeName': 'American Samoa',
  'numericCode': '016',
  'currencies': ['USD'],
  'languages': ['en', 'sm'],
  'translations': {'de': 'Amerikanisch-Samoa',
   'es': 'Samoa Americana',
   'fr': 'Samoa américaines',
   'ja': 'アメリカ領サモア',
   'it': 'Samoa Americane'},
  'relevance': '0.5'},
 {'_id': ObjectId('60bcb42e165a6db004093d6c'),
  'name': 'Bonaire',
  'topLevelDomain': ['.an', '.nl'],
  'alpha2Code': 'BQ',
  'alpha3Code': 'BES',
  'callingCodes': ['5997'],
  'capital': 'Kralendijk',
  'a